# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [11]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [12]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [13]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node a6ed14f2-7bc0-440e-955a-322ce3facf32 does not have a summary. Skipping filtering.
Node 1d4bb8bc-3456-45ed-94ef-f3676d6b69ad does not have a summary. Skipping filtering.
Node 565b2a5e-ae81-4fac-8f1c-7887d2970f71 does not have a summary. Skipping filtering.
Node 164be666-2eba-4683-84b9-883dcddce05b does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 138)

We can save and load our knowledge graphs as follows.

In [14]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 138)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [16]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* RAGAS employs a knowledge graph-based approach to process documents and create a test dataset. A SingleHopSpecificQuerySynthesizer, focused on single-hop queries, utilizes only one node from the graph(keyphrases, titles, or headlines) to generate a query. In this context, "single-hop" refers to the use of a single node from a knowledge graph built from the input document. Like the previous synthesizer, MultiHopAbstractQuerySynthesizer also utilizes specific properties. However, it achieves this by using multiple overlapping chunks. These would require the retrieval process to be able to retrieve multiple documents or sections to construct the desired context. MultiHopSpecificQuerySynthesizer aims to provide generalized (abstract) queries by utilizing multiple notes from the document.
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the main goal of regulating AI in the ...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The act aims to regulate the development and u...,single_hop_specifc_query_synthesizer
1,What are the concerns associated with Artifici...,[AI presents enormous opportunities for the Ph...,The concerns regarding ASI include the possibi...,single_hop_specifc_query_synthesizer
2,What is the REPLBLIC OF THE PHILIPPINES's poli...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The policy of the State is to recognize that s...,single_hop_specifc_query_synthesizer
3,How does the regulation of Artificial Superint...,"[1 \na) Promote innovation, technological adva...","The regulation of AI, including Artificial Sup...",single_hop_specifc_query_synthesizer
4,What is AGI in AI policy?,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial General Intelligence (AGI) refers t...,single_hop_specifc_query_synthesizer
5,"How does the regulation and oversight of AI, i...",[<1-hop>\n\n1 \nk) Issue advisory opinions on ...,The regulation and oversight framework establi...,multi_hop_abstract_query_synthesizer
6,How does the NAIC's responsibility for issuing...,[<1-hop>\n\n1 \nk) Issue advisory opinions on ...,The NAIC's responsibility for issuing advisory...,multi_hop_abstract_query_synthesizer
7,How does the NAIC's authority to impose admini...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The NAIC has the authority to impose administr...,multi_hop_abstract_query_synthesizer
8,How do Sec. 14 and Sec. 17 relate to AI market...,[<1-hop>\n\n1 \nSec. 12. Contents of AI Regist...,"Sec. 14 emphasizes that all AI systems used, i...",multi_hop_specific_query_synthesizer
9,CHED role in AI rules and CHED involved in AI ...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The context shows that CHED is part of the NAI...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 28c57c31-fc5a-4e1d-ae7d-7b1a5e0531e4 does not have a summary. Skipping filtering.
Node 364c303a-ddec-42de-b746-d4a24e43ea4b does not have a summary. Skipping filtering.
Node dae0cf36-9e01-404e-9529-9dda0a49abf1 does not have a summary. Skipping filtering.
Node c30b8149-3067-4ca2-a08d-b89745280766 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the European Union in AI r...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The European Union's landmark AI Act of 2024 h...,single_hop_specifc_query_synthesizer
1,Why Georgetown University important for AI pol...,[AI presents enormous opportunities for the Ph...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
2,WhAT is the role of the PHILIPPINES in regulat...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The Philippines has enacted the Artificial Int...,single_hop_specifc_query_synthesizer
3,What is Artificial Superintelligence?,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,"How do AI Foundation Models, like Large Langua...",[<1-hop>\n\n1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11...,"AI Foundation Models, including Large Language...",multi_hop_abstract_query_synthesizer
5,How do penalties for non-disclosure and mislab...,[<1-hop>\n\n1 \niii) Mandatory compliance trai...,The context specifies that any person who know...,multi_hop_abstract_query_synthesizer
6,how AI responses need verifiable info and resp...,[<1-hop>\n\n1 \nground its responses in verifi...,The context explains that AI responses should ...,multi_hop_abstract_query_synthesizer
7,How do the quasi-judicial powers of the NAIC r...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The NAIC has quasi-judicial powers to hear and...,multi_hop_abstract_query_synthesizer
8,How does the Senate's Artificial Intelligence ...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The Senate's Artificial Intelligence Regulatio...,multi_hop_specific_query_synthesizer
9,How do the development and regulation of Artif...,[<1-hop>\n\n1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11...,The development and regulation of Artificial G...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* The 'qa_evaluator' checks the correctness (correct/incorrect) by comparing the model’s (LLM) predicted answer (output) to the reference answer (ground truth).
* The 'labeled_helpfulness_evaluator' evaluates if the predicted answer (output) of the evaluation LLM is helpful to the user given the question and the reference answer using the labeled_criteria rubric.
</div>

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'essential-kiss-76' at:
https://smith.langchain.com/o/3331129d-b787-4202-99ff-e93635d9e432/datasets/3c79d119-9d88-4336-bbb0-990b5b2a83bf/compare?selectedSessions=2a4a65ef-c69f-4f51-a047-1ea5114a55c5




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the penalties related to AI development...,"Based on the provided context, the penalties r...",None,The context highlights that violations such as...,1,1,0,8.590880,9b84ad53-7d20-4c81-9b1b-d022aa9c9521,bd206c5b-b727-4d9d-a869-1846e9074d33
1,How does the DOST contribute to the developmen...,Based on the provided context:\n\nThe Departme...,None,The DOST contributes to AI policy development ...,1,0,0,5.616947,dba4f1fd-899a-4968-baf9-8cafe8096db4,1cc6457a-ea8e-4af6-8691-958668ed886c
2,How do the development and regulation of Artif...,The development and regulation of Artificial G...,None,The development and regulation of Artificial G...,1,1,0,7.871289,39c9e5c8-ac85-4a34-92cf-8c0fd238bbd2,05839b3e-bb0c-4a2a-9c9d-be86480f154f
3,How does the Senate's Artificial Intelligence ...,The Senate's Artificial Intelligence Regulatio...,None,The Senate's Artificial Intelligence Regulatio...,0,0,0,3.996056,534ef6e1-8097-40bf-98f2-f6381e313a2d,248219d3-f503-45c4-9ea5-11ddc41159c9
4,How do the quasi-judicial powers of the NAIC r...,The quasi-judicial powers of the NAIC enable i...,None,The NAIC has quasi-judicial powers to hear and...,1,0,0,3.037332,3a272745-d733-47e3-ab8d-8d50d561b0aa,fa72e606-86a8-4e34-85ee-f128ef40a292
5,how AI responses need verifiable info and resp...,"Based on the provided context, AI responses ne...",None,The context explains that AI responses should ...,1,1,0,4.961551,ce15ca2f-f696-4a41-8b40-de93f2467bd9,6e9986be-995c-49e0-83db-13f0880324ba
6,How do penalties for non-disclosure and mislab...,"Based on the provided context, the AI Ethics R...",None,The context specifies that any person who know...,1,1,0,6.330084,8689b774-6815-45e0-a854-3f02d3fb1e31,aaaa2524-3339-4edb-9775-8ba98695c6b6
7,"How do AI Foundation Models, like Large Langua...","Based on the provided context, AI Foundation M...",None,"AI Foundation Models, including Large Language...",1,1,0,5.681149,5f229efd-6634-4e90-96d4-e3ae824a630d,d0e256cb-f1aa-463c-b8be-d6bfca97547e
8,What is Artificial Superintelligence?,Artificial Superintelligence (ASI) refers to h...,None,The context states that the Act shall regulate...,0,1,0,3.764903,b2909075-bfe1-4cbc-a3ad-59e48d685156,07e9ecf8-470d-44c1-81fd-7e8c2ab15a19
9,WhAT is the role of the PHILIPPINES in regulat...,The role of the Philippines in regulating AI d...,None,The Philippines has enacted the Artificial Int...,1,1,0,3.482048,5448741f-ddd5-40c9-a03e-34be8d591afa,d217586f-43eb-4a2d-ac6d-d5a2ef583901


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:
Modifying the chunk size can significantly impact our application's performance in several ways: 
1. **Retrieval Quality or Relevance**: If chunks are too small, they may lack the necessary context, making it challenging for the retriever to deliver useful results. On the other hand, if the chunks are too large, they may include irrelevant content or combine semantically different ideas.
2. **Embedding Quality**: Small embeddings may not convey enough meaning, while larger embeddings can become "diluted" or less focused.
3. **Additional Factors**: Chunk size also affects LLM Input Context Limits, latency, and cost. Overall, finding the right balance in chunk size is essential for optimizing performance.
* Ultimately, the effectiveness of precision versus recall, as well as the responses of the LLM, relies on the quality of the retrieved context. If the chunking is off, even the most advanced LLM may produce hallucinations, incomplete answers, or irrelevant information.

</div>

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :
* Modifying the embedding model can dramatically affect the retrieval performance, accuracy, and overall quality of our LLM-based application, especially in RAG systems. Changing the embedding model changes how the text is understood and represented as vectors, directly affecting the retrieval of relevant information and the quality of our application's answers.
</div>

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for asking such an important question. Based on the context you've provided, the Philippines AI Bill is important because it aims to balance encouraging technological innovation with ensuring that AI systems remain safe, ethical, transparent, and under meaningful human oversight. This bill recognizes that AI is still in its early stages and seeks to provide a general framework for its responsible and lawful development.\n\nImportantly, the bill envisions a future where AI supports Filipino ingenuity, helps address national development challenges, and protects the rights and welfare of every citizen. It emphasizes the State’s responsibility to prevent AI from being used to commit crimes, abuse rights, or cause harm—whether intentionally or accidentally—while still promoting technological progress.\n\nIn essence, this bill is designed to guide the country's digital transformation in a way that is both innovative and mindful of safety, ethics, and the well-being of the Filipino

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'brief-decision-90' at:
https://smith.langchain.com/o/3331129d-b787-4202-99ff-e93635d9e432/datasets/3c79d119-9d88-4336-bbb0-990b5b2a83bf/compare?selectedSessions=4bb57829-129a-488e-becd-e94a11026553




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the penalties related to AI development...,Thank you for your thoughtful question. It’s c...,None,The context highlights that violations such as...,1,0,1,10.791690,9b84ad53-7d20-4c81-9b1b-d022aa9c9521,cef877cb-1500-4004-a519-8e0bcc49e7d1
1,How does the DOST contribute to the developmen...,Thank you for your thoughtful question. Based ...,None,The DOST contributes to AI policy development ...,1,1,1,9.069400,dba4f1fd-899a-4968-baf9-8cafe8096db4,60c4953b-daef-4d97-87dc-8994af786214
2,How do the development and regulation of Artif...,Thank you for your thoughtful question. From t...,None,The development and regulation of Artificial G...,1,1,1,8.541070,39c9e5c8-ac85-4a34-92cf-8c0fd238bbd2,f1098f1b-7d86-4849-a46a-d447a42aeb03
3,How does the Senate's Artificial Intelligence ...,Thank you for your thoughtful question about t...,None,The Senate's Artificial Intelligence Regulatio...,1,1,1,8.184660,534ef6e1-8097-40bf-98f2-f6381e313a2d,3215f6a2-515b-4334-ae5a-6d43ca20e0aa
4,How do the quasi-judicial powers of the NAIC r...,Thank you for your thoughtful question. Based ...,None,The NAIC has quasi-judicial powers to hear and...,1,1,1,7.499443,3a272745-d733-47e3-ab8d-8d50d561b0aa,5906f617-10c5-4ad6-9537-923933ced561
5,how AI responses need verifiable info and resp...,Thank you for your thoughtful question. Based ...,None,The context explains that AI responses should ...,1,1,1,11.152512,ce15ca2f-f696-4a41-8b40-de93f2467bd9,89c5f99e-269f-4f15-9c7e-0d728d65002f
6,How do penalties for non-disclosure and mislab...,Thank you for your thoughtful question. Based ...,None,The context specifies that any person who know...,1,1,1,7.474963,8689b774-6815-45e0-a854-3f02d3fb1e31,a79d40f6-088d-4490-9ec4-931d5d54b9f0
7,"How do AI Foundation Models, like Large Langua...",Thank you for your thoughtful question. Based ...,None,"AI Foundation Models, including Large Language...",1,1,1,10.341161,5f229efd-6634-4e90-96d4-e3ae824a630d,6ce8a7db-59a7-4ac5-9e6e-087f356d7562
8,What is Artificial Superintelligence?,Thank you for your question—it's important to ...,None,The context states that the Act shall regulate...,0,1,1,2.753457,b2909075-bfe1-4cbc-a3ad-59e48d685156,d1584d4a-e8c5-4826-bfdf-7b96e91a3d0b
9,WhAT is the role of the PHILIPPINES in regulat...,Thank you for your thoughtful question. Based ...,None,The Philippines has enacted the Artificial Int...,1,1,1,3.069167,5448741f-ddd5-40c9-a03e-34be8d591afa,eb0a6352-01e0-429b-8481-ae8a2b427b5f


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:
I believe that certain metrics have changed in specific ways due to the modifications we implemented. 

**Correctness**: Larger chunks of information reduced boundary misses, and stronger embeddings, such as “text-embedding-3-large,” improved semantic recall. 

**Helpfulness**: Enhanced grounding and clearer answers resulted from having a more complete context. 

**Empathy**: The new prompt explicitly instructs the use of an empathetic tone, which directly optimizes the “empathy” criterion. 

* First Chain

![First Chain](First%20Chain.png)

* Second Chain

![Second Chain](Second%20Chain.png)
</div>